# Geospatial Queries

This notebook demonstrates how to use SurrealDB's geospatial features through the ORM to build location-aware applications like restaurant finders, store locators, and delivery zone checkers.

SurrealDB natively supports GeoJSON geometry types, enabling proximity searches and distance calculations without external geospatial extensions.

## Use Case: Restaurant Finder

We'll build a restaurant finder that lets users:
1. Search for restaurants near a location
2. Sort results by distance
3. Filter by cuisine type and proximity

## Prerequisites

- SurrealDB running locally (`docker run --rm -p 8000:8000 surrealdb/surrealdb:latest start --user root --pass root`)
- Project dependencies installed (`uv sync`)
- A `.env` file in the project root (optional, falls back to defaults)

In [ ]:
# Setup: add project root to path and configure SurrealDB connection
import os, sys
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)
from dotenv import load_dotenv
load_dotenv()

from src.surreal_orm import SurrealDBConnectionManager

SurrealDBConnectionManager.set_connection(
    os.getenv("SURREALDB_URL", "ws://localhost:8000"),
    os.getenv("SURREALDB_USER", "root"),
    os.getenv("SURREALDB_PASS", "root"),
    os.getenv("SURREALDB_NAMESPACE", "ns"),
    os.getenv("SURREALDB_DATABASE", "db"),
)

## 1. Defining Geospatial Models

The ORM provides typed geometry fields that map to SurrealDB's `geometry<point>`, `geometry<polygon>`, etc. These fields store GeoJSON objects.

In [ ]:
# Define models with geospatial fields
from src.surreal_orm import BaseSurrealModel, SurrealConfigDict
from src.surreal_orm.fields import PointField, PolygonField
from src.surreal_orm.geo import GeoDistance

class Restaurant(BaseSurrealModel):
    model_config = SurrealConfigDict(table_name="restaurant")

    id: str | None = None
    name: str
    cuisine: str = ""
    rating: float = 0.0
    location: PointField  # Maps to geometry<point> in SurrealDB

In [ ]:
# Create restaurants with GeoJSON Point data
# IMPORTANT: GeoJSON uses (longitude, latitude) order -- NOT (lat, lon)!
# These coordinates are around Midtown Manhattan, New York City
restaurants = [
    Restaurant(
        name="Pizza Palace",
        cuisine="Italian",
        rating=4.5,
        location={"type": "Point", "coordinates": [-73.985, 40.748]},
    ),
    Restaurant(
        name="Sushi Supreme",
        cuisine="Japanese",
        rating=4.8,
        location={"type": "Point", "coordinates": [-73.978, 40.752]},
    ),
    Restaurant(
        name="Taco Town",
        cuisine="Mexican",
        rating=4.2,
        location={"type": "Point", "coordinates": [-73.990, 40.745]},
    ),
    Restaurant(
        name="Burger Barn",
        cuisine="American",
        rating=4.0,
        location={"type": "Point", "coordinates": [-73.970, 40.760]},
    ),
    Restaurant(
        name="Pad Thai Place",
        cuisine="Thai",
        rating=4.6,
        location={"type": "Point", "coordinates": [-73.982, 40.750]},
    ),
    Restaurant(
        name="Dim Sum House",
        cuisine="Chinese",
        rating=4.3,
        location={"type": "Point", "coordinates": [-74.000, 40.720]},  # Farther away (Chinatown)
    ),
]

for r in restaurants:
    await r.save()
    print(f"Saved: {r.name} ({r.cuisine}) at {r.location['coordinates']}")

## 2. GeoField Types Reference

The ORM provides several geometry field types, each mapping to a SurrealDB geometry subtype:

| Python Field | SurrealDB Type | GeoJSON Type | Use Case |
|---|---|---|---|
| `PointField` | `geometry<point>` | Point | Locations (restaurants, users) |
| `PolygonField` | `geometry<polygon>` | Polygon | Areas (delivery zones, regions) |
| `LineStringField` | `geometry<linestring>` | LineString | Routes, paths |
| `MultiPointField` | `geometry<multipoint>` | MultiPoint | Clusters of locations |
| `GeoField["polygon"]` | `geometry<polygon>` | Polygon | Generic form (same as PolygonField) |

You can also use `GeoField["point"]` as an alternative to `PointField` -- they are equivalent.

## 3. Nearby Search (Proximity)

The `nearby()` method finds records within a given distance from a reference point. This is the most common geospatial query -- "what's near me?"

In [ ]:
# Find restaurants within 5km of Times Square
# Point: (longitude, latitude) = (-73.9855, 40.7580)
times_square = (-73.9855, 40.7580)

nearby = await Restaurant.objects().nearby(
    "location", times_square, max_distance=5000  # 5000 meters = 5km
).exec()

print(f"Restaurants within 5km of Times Square ({len(nearby)} found):")
for r in nearby:
    print(f"  - {r.name} ({r.cuisine})")

In [ ]:
# Combine nearby search with other filters
# Find only Italian restaurants within 5km
nearby_italian = await Restaurant.objects().filter(
    cuisine="Italian"
).nearby(
    "location", times_square, max_distance=5000
).exec()

print(f"Italian restaurants near Times Square ({len(nearby_italian)} found):")
for r in nearby_italian:
    print(f"  - {r.name} (rating: {r.rating})")

## 4. Distance Annotations

Use `GeoDistance` to compute the distance from each record to a reference point. This lets you sort by distance and display "X km away" in your UI. The `GeoDistance` helper uses the same `annotate()` interface as `SearchScore`.

In [ ]:
# Annotate each restaurant with its distance from Times Square
# Then sort by distance (closest first) and take the top 5
results = await Restaurant.objects().annotate(
    dist=GeoDistance("location", times_square),
).order_by("dist").limit(5).exec()

print("Nearest 5 restaurants to Times Square:")
for r in results:
    distance = getattr(r, "dist", "N/A")
    print(f"  - {r.name} ({r.cuisine}) -- distance: {distance}")

In [ ]:
# Combine distance annotation with rating filter
# Find highly-rated restaurants sorted by proximity
results = await Restaurant.objects().filter(
    rating__gte=4.5
).annotate(
    dist=GeoDistance("location", times_square),
).order_by("dist").exec()

print("Top-rated restaurants (>= 4.5) sorted by distance:")
for r in results:
    distance = getattr(r, "dist", "N/A")
    print(f"  - {r.name} (rating: {r.rating}, distance: {distance})")

## Important: Coordinate Order

GeoJSON uses **(longitude, latitude)** order -- this is the opposite of what most people expect (Google Maps uses lat/lon).

| Format | Order | Example (NYC) |
|---|---|---|
| GeoJSON | (longitude, latitude) | (-73.985, 40.748) |
| Google Maps | (latitude, longitude) | (40.748, -73.985) |

Getting this wrong will place your points on the wrong side of the planet. Always double-check!

In [ ]:
# Cleanup: remove all test data
await Restaurant.objects().delete_table()
print("Cleanup complete.")